In [44]:
%load_ext autoreload
%autoreload 2

from tweedejaars_project import load_df
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import *
from sklearn.tree import *
from sklearn.metrics import *
from sklearn.linear_model import *


df = load_df()

print(df)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
                        datetime  import_capacity  upward_dispatch_published  \
0      2023-12-29 00:00:00+00:00             65.4                        0.0   
1      2023-12-29 00:01:00+00:00             65.4                        0.0   
2      2023-12-29 00:02:00+00:00             65.4                        0.0   
3      2023-12-29 00:03:00+00:00             65.4                        0.0   
4      2023-12-29 00:04:00+00:00             65.4                        0.0   
...                          ...              ...                        ...   
220241 2024-05-29 22:41:00+00:00           1982.6                      167.0   
220242 2024-05-29 22:42:00+00:00           1982.6                      165.0   
220243 2024-05-29 22:43:00+00:00           1982.6                      162.0   
220244 2024-05-29 22:44:00+00:00           1982.6                      158.0   
220245 2024-05-29 22:45:00+00:00

In [19]:
features = [
    "import_capacity",
    "settlement_price_bestguess",
]

train_x = df[features][:165195]
train_y = df['target_two_sided_ptu'][:165195]

test_x = df[features][165195:]
test_y = df['target_two_sided_ptu'][165195:]

# Initialize the Decision Tree Classifier
clf = HistGradientBoostingClassifier()

# Train the classifier
clf.fit(train_x, train_y)

y_pred = clf.predict(test_x)

# Evaluate the model
accuracy = accuracy_score(test_y, y_pred)
print(accuracy)

print("Classification Report:")
print(classification_report(test_y, y_pred))

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(test_y, y_pred))

0.9785290003814645
Classification Report:
              precision    recall  f1-score   support

       False       0.98      1.00      0.99     54028
        True       0.16      0.04      0.06      1023

    accuracy                           0.98     55051
   macro avg       0.57      0.52      0.53     55051
weighted avg       0.97      0.98      0.97     55051

Confusion Matrix:
[[53830   198]
 [  984    39]]


In [91]:
def compute_penalty(df: pd.DataFrame, true, pred, ids, example_revenue=False):
    energy = 100 / 60 # Example renewable energy
    df = df.copy()
    df['min_price'] = df['settlement_price_bestguess']
    df['max_price'] = df['settlement_price_bestguess']
    df['pred'] = pred
    df['id'] = ids
    df['true'] = true
    # False pos and false neg in respect to naive
    df['false_neg'] = False
    df['false_pos'] = True
    df.loc[~df['pred'], 'false_neg'] = df.loc[~df['pred'], 'naive_strategy_action']
    df.loc[~df['true'], 'false_pos'] = ~df.loc[~df['true'], 'pred'] & df.loc[~df['true'], 'naive_strategy_action']
    agg_dict = {
        'min_price': 'min', # Min down price
        'max_price': 'max', # Max up price
        'false_neg': 'sum', # Count all the false negatives
        'false_pos': 'sum', # Count all the false positives
        'true': 'any' # Is PTU two-sided
    }
    flat_df = df.groupby('id').aggregate(agg_dict)
    false_neg_view = flat_df[flat_df['true']]
    false_pos_view = flat_df[~flat_df['true']]
    
    if not example_revenue:
        return false_neg_view['false_neg'].sum(), false_pos_view['false_pos'].sum()
    
    false_neg_penalty = energy * false_neg_view['max_price'] * false_neg_view['false_neg']
    false_pos_penalty = energy * false_pos_view['min_price'] * false_pos_view['false_pos']
    
    return -false_neg_penalty.sum(), -false_pos_penalty.sum()

# def compute_penalty(df: pd.DataFrame, true, pred, ids):
#     energy = 100 / 60 # Example renewable energy
#     df = df.copy()
#     df['min_price'] = df['settlement_price_bestguess']
#     df['max_price'] = df['settlement_price_bestguess']
#     df['pred'] = pred
#     df['id'] = ids
#     df['true'] = true
#     # False pos and false neg in respect to naive
#     df['outcome'] = False
#     df.loc[~df['pred'], 'outcome'] = df.loc[~df['pred'], 'naive_strategy_action']
#     agg_dict = {
#         'min_price': 'min', # Min down price
#         'max_price': 'max', # Max up price
#         'outcome': 'sum', # Count all the false negatives
#         'true': 'any' # Is PTU two-sided
#     }
#     flat_df = df.groupby('id').aggregate(agg_dict)
#     print(flat_df)
#     false_neg_view = flat_df[flat_df['true']]
#     false_pos_view = flat_df[~flat_df['true']]
#     false_neg_penalty = energy * false_neg_view['max_price'] * false_neg_view['false_neg']
#     false_pos_penalty = energy * false_pos_view['min_price'] * (15 - false_pos_view['false_pos'])
    
#     return false_neg_penalty.sum(), false_pos_penalty.sum()

compute_penalty(df, df['target_two_sided_ptu'], df['target_two_sided_ptu'], df['ptu_id'])

(0, 20384)

In [92]:
false_neg_test_df = {
    'Time': [
        '11:30', '11:31', '11:32', '11:33', '11:34', '11:35', '11:36', 
        '11:37', '11:38', '11:39', '11:40', '11:41', '11:42', '11:43', 
        '11:44', '11:45'
    ],
    'min_price_published': [
        -100, np.NaN, 0, -100, -150, 
        -200, -300, -100, np.NaN, np.NaN, 
        np.NaN, np.NaN, np.NaN, -30, -100, 0
    ],
    'max_price_published': [
        np.NaN, 20, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, 
        40, 50, 90, 200, 80, 
        np.NaN, np.NaN, np.NaN
    ],
    'settlement_price_bestguess': [
        np.NaN, np.NaN, 0, -100, -150, -200, -300, -300, 
        40, 50, 90, 200, 200, 
        200, 200, 200
    ],
    'target_two_sided_ptu': [
        False, False, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True
    ],
    'pred': [
        True, False, False, True, True, False, False, True, False, False,
        False, False, False, False, False, False
    ],
    'naive_strategy_action': [
        False, False, False, True, True, True, True, True, False, False,
        False, False, False, False, False, False,
    ],
    'ptu_id':
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
}

false_pos_test_df = {
    'Time': [
        '11:30', '11:31', '11:32', '11:33', '11:34', '11:35', '11:36', 
        '11:37', '11:38', '11:39', '11:40', '11:41', '11:42', '11:43', 
        '11:44', '11:45'
    ],
    'min_price_published': [
        -100, -100, 0, -100, -150, 
        -200, -300, -100, -100, -100, 
        -100, -100, -100, -30, -100, 0
    ],
    'max_price_published': [
        np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, 
        np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, 
        np.NaN, np.NaN, np.NaN
    ],
    'settlement_price_bestguess': [
        np.NaN, np.NaN, 0, -100, -150, -200, -300, -300, 
        -300, -300, -300, -300, -300, 
        -300, -300, -300
    ],
    'target_two_sided_ptu': [
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False
    ],
    'pred': [
        True, False, False, True, True, False, False, True, False, False,
        False, False, False, False, False, False
    ],
    'naive_strategy_action': [
        False, False, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True,
    ],
    'ptu_id':
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
}

false_neg_test_df = pd.DataFrame(false_neg_test_df)
false_pos_test_df = pd.DataFrame(false_pos_test_df)

false_neg = compute_penalty(false_neg_test_df, false_neg_test_df['target_two_sided_ptu'], false_neg_test_df['pred'], false_neg_test_df['ptu_id'])
false_pos = compute_penalty(false_pos_test_df, false_pos_test_df['target_two_sided_ptu'], false_pos_test_df['pred'], false_pos_test_df['ptu_id'])
print(false_neg)
print(false_pos)

(2, 0)
(0, 11)
